# Coursera IBM Data Science Capstone Project
## Segmenting and Clustering Neighborhods in Toronto

Creating the prescribed data-frame with the Postal-Code, Borough, and Neighborhood using the provided Wikipedia link:

In [1]:
import pandas as pd
import numpy as np
import folium
import json
import requests
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Reading in the table from the provided link and checking to see that it is indeed a Pandas data-frame:

In [2]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)[0]
dfs.head()

,Post Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
type(dfs)

pandas.core.frame.DataFrame

### Table for Part 1

We need to remove the 'Not assigned' rows from the table and change the column header for 'Postal Code' to 'PostalCode':

In [4]:
df = dfs[dfs.Borough != 'Not assigned'].reset_index(drop = True)
df = df.rename(columns = {"Post Code" : "PostalCode"})
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


The shape of the above array is given by:

In [7]:
df.shape

(103, 3)

Now, we need to include the coordinates for each of the postal codes by appending two columns for the lattitude and longidude (the geocoder library did not work):

### Table for Part 2:

In [8]:
url = "https://cocl.us/Geospatial_data"
neigh_coords = pd.read_csv(url)
neigh_coords = neigh_coords.rename(columns = {"Postal Code" : "PostalCode"})

In [9]:
df_geo = pd.merge(left = df, right = neigh_coords, on = 'PostalCode' )
df_geo.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [10]:
df_geo.shape

(103, 5)

### Map of the Neighborhoods:
Now, we can make a map of the the Boroughs and the neighborhoods that they contain in Toronto:

In [42]:
map_toronto = folium.Map(location = [43.7, -79.4], zoom_start = 11)

for lat, lng, borough, neighborhood in zip(df_geo['Latitude'], df_geo['Longitude'], df_geo['Borough'], df_geo['Neighborhood']):
    label = '{}: {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Notice that the distance between neighborhoods is not the same among all neighborhoods, namely downtown neighborhoods are closer together than suburban neighborhoods. When calling the Foursquare API it will ask for a distance from the center of the geographic coordinates. In order to account for this change in spacing, we can set the max radius as the distance between the closest adjacent neighborhood. This will minimize the amount of overlap in venues among closely packed neighborhoods, and maximize the amount of venues in subruban venues, when compared to using a fixed radius. The next few line of code do this, and we create the new data-frame with this minimim radius included for analysis in the future.

In [12]:
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from vincenty import vincenty

dist_matrix = pdist(neigh_coords[['Latitude','Longitude']], lambda u, v: vincenty(u,v))
dist_matrix = squareform(dist_matrix)
np.fill_diagonal(dist_matrix, 100000000)
min_dist = dist_matrix.min(axis = 1) * 1000
min_dist = pd.DataFrame(min_dist, columns = ['MinDist'])
min_dist.head()

,MinDist
0,3394.464
1,3253.397
2,2416.097
3,1831.814
4,1831.814


In [13]:
df_geo_full = pd.concat([df_geo, min_dist], axis = 1)

In [14]:
df_geo_full.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,MinDist
0,M3A,North York,Parkwoods,43.753259,-79.329656,3394.464
1,M4A,North York,Victoria Village,43.725882,-79.315572,3253.397
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2416.097
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1831.814
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1831.814


It looks like we have the appropraite geographic information for the API. Note that the `MinDist` is in meters.

Now, we use the Foursquare API to obtain data about various places in the neighborhoods/postal codes in Toronto:

In [37]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


Let's perform a test run using the API on the first neighborhood:

In [16]:
neighborhood_latitude = df_geo.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_geo.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_geo.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [40]:
LIMIT = 10 
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [18]:
results = requests.get(url).json()

We can ues the `get_category_type` function introduced in the lab assingment to aid in parsing the `.json` file into a Pandas data-frame:

In [19]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

The code used in the lab is applied heire to convert the venue name and relavent information into a Pandas dataframe:

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,TTC stop - 44 Valley Woods,Bus Stop,43.755402,-79.333741


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


All looks well! Once again, we can use the `getNearbyVenues` function introduced in the lab assingment to obtain the venues in each of the neighborhoods (prescriped by their geographic coordinates) using the API:

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng, radius in zip(names, latitudes, longitudes, radius):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
toronto_venues = getNearbyVenues(names=df_geo_full['Neighborhood'],
                                   latitudes=df_geo_full['Latitude'],
                                   longitudes=df_geo_full['Longitude'],
                                   radius = df_geo_full['MinDist']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(939, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,Parkwoods,43.753259,-79.329656,Galleria Supermarket,43.753520,-79.349518,Supermarket
3,Parkwoods,43.753259,-79.329656,Graydon Hall Manor,43.763923,-79.342961,Event Space
4,Parkwoods,43.753259,-79.329656,Naan & Kabob Halal,43.742903,-79.305148,Middle Eastern Restaurant


Onehot encoding is applied to the venue types:

In [25]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = "", prefix_sep = "")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo Exhibit,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Xinjiang Restaurant,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo Exhibit,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,American Restaurant,Art Gallery,Arts & Crafts Store,...,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Xinjiang Restaurant,Yoga Studio,Zoo
0,Agincourt,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
5,Berczy Park,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
6,"Birch Cliff, Cliffside West",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
7,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.100000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
8,"Business reply mail Processing Centre, South C...",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0,0.0,0.100000,0.1,0.1,0.0,0.0,0.000000,...,0.1,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


The top five venues for each neighborhood can be determined (using code from the lab):

In [27]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                   venue  freq
0   Caribbean Restaurant   0.2
1   Gym / Fitness Center   0.1
2      Indian Restaurant   0.1
3  Sri Lankan Restaurant   0.1
4            Supermarket   0.1


----Alderwood, Long Branch----
                       venue  freq
0         Seafood Restaurant   0.1
1  South American Restaurant   0.1
2                  Bookstore   0.1
3                Coffee Shop   0.1
4              Grocery Store   0.1


----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0    Coffee Shop   0.2
1           Bank   0.1
2     Restaurant   0.1
3  Deli / Bodega   0.1
4           Park   0.1


----Bayview Village----
                 venue  freq
0  Japanese Restaurant   0.2
1          Gas Station   0.2
2                 Bank   0.2
3        Grocery Store   0.1
4           Restaurant   0.1


----Bedford Park, Lawrence Manor East----
                     venue  freq
0              Coffee Shop   0.1
1        Indian Restaurant   0.1
2  Comfort Foo

                venue  freq
0         Golf Course   0.2
1    Toy / Game Store   0.1
2  Italian Restaurant   0.1
3          Restaurant   0.1
4    Tapas Restaurant   0.1


----Kennedy Park, Ionview, East Birchmount Park----
                venue  freq
0         Coffee Shop   0.3
1            Pharmacy   0.1
2  Chinese Restaurant   0.1
3    Department Store   0.1
4      Sandwich Place   0.1


----Kensington Market, Chinatown, Grange Park----
                  venue  freq
0           Record Shop   0.1
1   Arts & Crafts Store   0.1
2          Cocktail Bar   0.1
3  Caribbean Restaurant   0.1
4                  Café   0.1


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                venue  freq
0       Smoothie Shop   0.1
1  Chinese Restaurant   0.1
2      Sandwich Place   0.1
3                Bank   0.1
4         Gas Station   0.1


----Lawrence Manor, Lawrence Heights----
                    venue  freq
0              Restaurant   0.2
1  Furniture / Home S

4           Brewery   0.1


----The Kingsway, Montgomery Road, Old Mill North----
              venue  freq
0         Pool Hall   0.1
1      Dessert Shop   0.1
2      Liquor Store   0.1
3      Burger Joint   0.1
4  Sushi Restaurant   0.1


----Thorncliffe Park----
               venue  freq
0  Fish & Chips Shop   0.1
1        Coffee Shop   0.1
2            Brewery   0.1
3             Bakery   0.1
4  Indian Restaurant   0.1


----Toronto Dominion Centre, Design Exchange----
         venue  freq
0  Coffee Shop   0.2
1       Bakery   0.1
2          Gym   0.1
3        Hotel   0.1
4          Pub   0.1


----University of Toronto, Harbord----
              venue  freq
0      Concert Hall   0.1
1  Sushi Restaurant   0.1
2               Bar   0.1
3          Beer Bar   0.1
4              Park   0.1


----Upper Rouge----
          venue  freq
0         Trail  0.25
1    Playground  0.25
2   Golf Course  0.25
3          Farm  0.25
4  Noodle House  0.00


----Victoria Village----
                  

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Caribbean Restaurant,Gym / Fitness Center,Breakfast Spot,Sri Lankan Restaurant,Chinese Restaurant,Cantonese Restaurant,Supermarket,Bakery,Indian Restaurant,Design Studio
1,"Alderwood, Long Branch",Grocery Store,Bookstore,New American Restaurant,Café,Burger Joint,Seafood Restaurant,South American Restaurant,Coffee Shop,Pizza Place,Gourmet Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Restaurant,Mediterranean Restaurant,Ski Chalet,Middle Eastern Restaurant,Bridal Shop,Bank,Deli / Bodega,Park,Creperie
3,Bayview Village,Bank,Gas Station,Japanese Restaurant,Café,Grocery Store,Chinese Restaurant,Restaurant,Doner Restaurant,Dog Run,Discount Store
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Comfort Food Restaurant,Juice Bar,Pub,Italian Restaurant,Café,Restaurant,Sushi Restaurant,Indian Restaurant,Thai Restaurant


### Clustering the Neighborhoods

In [30]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10]

array([2, 1, 3, 2, 1, 2, 1, 3, 0, 0])

In [31]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [32]:
toronto_merged = df_geo
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged.dropna()
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head(7)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Coffee Shop,Caribbean Restaurant,Event Space,Middle Eastern Restaurant,Supermarket,Italian Restaurant,Bagel Shop,Mediterranean Restaurant,Gym,Golf Course
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Middle Eastern Restaurant,Gym / Fitness Center,History Museum,Mediterranean Restaurant,Indian Restaurant,Thai Restaurant,Hockey Arena,Rock Climbing Spot,Grocery Store,Creperie
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Park,Mediterranean Restaurant,Bakery,Farmers Market,Historic Site,Dessert Shop,Eastern European Restaurant,Doner Restaurant,Dog Run
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2,Restaurant,Greek Restaurant,Furniture / Home Store,Dessert Shop,Sushi Restaurant,Athletics & Sports,Fried Chicken Joint,Boutique,Vietnamese Restaurant,Cosmetics Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Park,Bubble Tea Shop,Sandwich Place,Creperie,Arts & Crafts Store,Dance Studio,Yoga Studio,Convenience Store,Cosmetics Shop
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,1,Golf Course,Restaurant,French Restaurant,Bakery,Café,Toy / Game Store,Italian Restaurant,Tapas Restaurant,Garden,Dance Studio
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2,Zoo Exhibit,Zoo,Athletics & Sports,Spa,Deli / Bodega,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run,Discount Store


### Map of the Clustered Neighborhoods:

In [39]:
# create map
map_clusters = folium.Map(location = [43.7, -79.4], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Clusters 0, 1, and 2 are the most common as indicated on the map above. We can examin the features of each of these cluseters to see if there are any differences among the venues:

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,0,Coffee Shop,Park,Bubble Tea Shop,Sandwich Place,Creperie,Arts & Crafts Store,Dance Studio,Yoga Studio,Convenience Store,Cosmetics Shop
8,East York,0,Gym / Fitness Center,Park,Bakery,Rock Climbing Spot,Restaurant,Brewery,Convenience Store,Gastropub,Pastry Shop,Indian Restaurant
12,Scarborough,0,Park,Grocery Store,Breakfast Spot,Burger Joint,Italian Restaurant,Athletics & Sports,Campground,Bakery,Zoo,Dessert Shop
14,East York,0,Rock Climbing Spot,Sandwich Place,Pastry Shop,Park,Gastropub,Ethiopian Restaurant,American Restaurant,Breakfast Spot,Café,Butcher
18,Scarborough,0,Park,Food & Drink Shop,Pizza Place,Smoothie Shop,Athletics & Sports,Fried Chicken Joint,Liquor Store,Pub,Golf Course,Concert Hall
19,East Toronto,0,Bakery,Trail,Bagel Shop,Ice Cream Shop,Gastropub,Park,Vegetarian / Vegan Restaurant,Health Food Store,Indie Movie Theater,Toy / Game Store
24,Downtown Toronto,0,Plaza,Historic Site,Park,Pizza Place,Supermarket,Theater,Art Gallery,Clothing Store,Gastropub,Garden
36,Downtown Toronto,0,Park,Plaza,Hotel,Performing Arts Venue,Salad Place,Dessert Shop,Lake,Sporting Goods Shop,Fried Chicken Joint,French Restaurant
45,North York,0,Pool,Park,Tennis Court,Business Service,Deli / Bodega,Eastern European Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner
47,East Toronto,0,Brewery,Gym,Ice Cream Shop,Farmers Market,Fast Food Restaurant,Park,Fish & Chips Shop,Asian Restaurant,Indian Restaurant,Doner Restaurant


In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Coffee Shop,Caribbean Restaurant,Event Space,Middle Eastern Restaurant,Supermarket,Italian Restaurant,Bagel Shop,Mediterranean Restaurant,Gym,Golf Course
1,North York,2,Middle Eastern Restaurant,Gym / Fitness Center,History Museum,Mediterranean Restaurant,Indian Restaurant,Thai Restaurant,Hockey Arena,Rock Climbing Spot,Grocery Store,Creperie
3,North York,2,Restaurant,Greek Restaurant,Furniture / Home Store,Dessert Shop,Sushi Restaurant,Athletics & Sports,Fried Chicken Joint,Boutique,Vietnamese Restaurant,Cosmetics Shop
6,Scarborough,2,Zoo Exhibit,Zoo,Athletics & Sports,Spa,Deli / Bodega,Electronics Store,Eastern European Restaurant,Doner Restaurant,Dog Run,Discount Store
10,North York,2,Clothing Store,Fried Chicken Joint,Playground,Greek Restaurant,Caribbean Restaurant,Restaurant,Shopping Mall,Bike Shop,Japanese Restaurant,Latin American Restaurant
15,Downtown Toronto,2,Food Truck,Gym,Coffee Shop,Middle Eastern Restaurant,Farmers Market,American Restaurant,Japanese Restaurant,Creperie,Restaurant,Theater
20,Downtown Toronto,2,Farmers Market,Museum,Japanese Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant,Basketball Stadium,Food Truck,Restaurant,Liquor Store,Zoo
27,North York,2,Chinese Restaurant,Cantonese Restaurant,Golf Course,Grocery Store,Korean Restaurant,Bubble Tea Shop,Hotel,BBQ Joint,Bakery,Design Studio
30,Downtown Toronto,2,Gym / Fitness Center,Thai Restaurant,Gym,Hotel,Mediterranean Restaurant,Pizza Place,Speakeasy,Plaza,American Restaurant,Art Gallery
33,North York,2,Chocolate Shop,Breakfast Spot,Shopping Mall,Caribbean Restaurant,American Restaurant,Middle Eastern Restaurant,Movie Theater,Tea Room,Toy / Game Store,Salon / Barbershop


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,3,Coffee Shop,Park,Mediterranean Restaurant,Bakery,Farmers Market,Historic Site,Dessert Shop,Eastern European Restaurant,Doner Restaurant,Dog Run
11,Etobicoke,3,Grocery Store,Fish & Chips Shop,Coffee Shop,Pub,Restaurant,Burger Joint,Pharmacy,Farmers Market,Sporting Goods Shop,Turkish Restaurant
17,Etobicoke,3,Coffee Shop,Grocery Store,Golf Course,Liquor Store,Racetrack,Park,Beer Store,Farmers Market,Flower Shop,Zoo
22,Scarborough,3,Coffee Shop,Music Store,Pharmacy,Gym,Juice Bar,Park,Restaurant,Burger Joint,Indian Restaurant,Dog Run
23,East York,3,Gym,BBQ Joint,Brewery,Liquor Store,Sporting Goods Shop,Sports Bar,Fish & Chips Shop,Bakery,Bike Shop,Coffee Shop
26,Scarborough,3,Coffee Shop,Caribbean Restaurant,Pharmacy,Sushi Restaurant,Burger Joint,Indian Restaurant,Music Store,Restaurant,Xinjiang Restaurant,Hakka Restaurant
28,North York,3,Coffee Shop,Restaurant,Mediterranean Restaurant,Ski Chalet,Middle Eastern Restaurant,Bridal Shop,Bank,Deli / Bodega,Park,Creperie
29,East York,3,Gym,BBQ Joint,Brewery,Liquor Store,Fish & Chips Shop,Indian Restaurant,Bakery,Coffee Shop,Warehouse Store,Afghan Restaurant
32,Scarborough,3,Ice Cream Shop,Sandwich Place,Coffee Shop,Liquor Store,Pizza Place,Indian Restaurant,Bookstore,Xinjiang Restaurant,Convenience Store,Design Studio
34,North York,3,Coffee Shop,Middle Eastern Restaurant,Caribbean Restaurant,Massage Studio,Italian Restaurant,Music Store,Restaurant,Pizza Place,Cosmetics Shop,Dessert Shop


Lets compare and contrase the first and second most common venues for each cluster. This is just a cursoury examination. Cluster 0 tends to focus more on more social and casual venues, i.e. casual dining and parks. Cluster 1 tends to have more diverse dining options and a greater emphasis on cultural activities. Cluster 2 clearly has the greatest emphasis on coffee shops fast-casual dining and some retail oppertunities. Cluster 0 would be good for a person who enjoys the outdoors and being in social situations. Cluster 1 would be good for a person who enjoys multi-cultural dining options and museaums/other cultural experiences. Cluster 2 would be good for someone who like coffee shops or who works from home and occasionally enjoyes working in coffee shops. Futher feature engineering could be performed, but at this stage we are seeing distinct characteristics emerge that describe each of the neighborhoods. 